In [1]:
import re
import nltk
import random

from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install tweepy

# TextBlob: textblob is the python library for processing textual data.

# Install it using following pip command:
!pip install textblob


#Also, we need to install some corpora using following command:
!python -m textblob.download_corpora

#(Corpora is nothing but a large and structured set of texts)

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!

In [3]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
from nltk.corpus import sentiwordnet as swn
  
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    We create a TwitterClient class. This class contains all the methods to interact with Twitter API and parsing tweets. 
    We use __init__ function to handle the authentication of API client.
    
    '''
    def __init__(self): 
        
        # __init__ is the constructor for a class. The self variable represents the instance of the object itself.
        # When a class defines an __init__() method, class instantiation automatically invokes __init__() for the newly-created class instance. 
        
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        
        consumer_key = 'I5EQI9SZPHbzHU1NY1KK3C2qx'
        consumer_secret = 'OOsThzGqX1niE9YN6hPwoTG9Q4u5rij3mqsIuOwg9VsZve5LYR'
        access_token = '147830710-w3n86zZ3f69z9xX8XfACEKAlCq8uRaFU7lFVGBM6'
        access_token_secret = '9oTYADmKqvPXLH5pnO9tP3pBlN6mmAZEAbDn0kPAOznSD'
        
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 

        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return(' '.join(re.sub("([,\.():;!$%^&*\d])|([^0-9A-Za-z \t])", " ", tweet).split())) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
       
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return('positive')
        elif analysis.sentiment.polarity == 0: 
            return ('neutral')
        else: 
            return ('negative')
        


  
    def get_tweets(self, query, count = 100): 
        ''' 
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets 
        tweets = [] 
        
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
              
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text
                parsed_tweet['date'] = tweet.created_at
                
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return (tweets) 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 
  

In [6]:
def main(): 
    
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'work from home covid', count = 1500) 
    
 
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} % ".format(100*(len(tweets) - (len(ntweets) + len(ptweets)))/len(tweets))) 
  
    # printing first 10 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text'])
        # print(tweet['date'])
  
    # printing first 10 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text'])
        # print(tweet['date'])
    

In [7]:
if __name__ == "__main__": 
    # calling main function 
    main() 

Positive tweets percentage: 49.0 %
Negative tweets percentage: 11.0 %
Neutral tweets percentage: 40.0 % 


Positive tweets:
📣 New Podcast! "Earth911 Podcast: Making environmental lemonade from COVID-19 lemons" on @Spreaker #cfls #changes… https://t.co/ThiWDgMlIw
Wo rking from home is a solution for Small Businesses amid COVID-19 Social Distancing. Read more:… https://t.co/H9xgYmikg5
RT @niawag2011: Been thinking about this a lot. In a study in the Mission District in SF, no caucasians tested positive. Almost all positiv…
@adamvaughan_uk Pre-covid would be appealing, post covid think I’ll work from home far more often as will many I feel.
RT @Rollits: Is your workplace "covid-secure"?  With the recent Government announcement encouraging those who can't work from home to go ba…
RT @FarmersOfTheUK: I cover a lovely area of Yorkshire including the North Yorkshire Moors and Yorkshire Wolds. I’m home based so Covid has…
RT @TaP_Arden: Join us on the 28th May at 3pm for a celebration of work 

In [16]:
from nltk.corpus import sentiwordnet as swn
breakdown = swn.senti_synset('work.v.03')
print(breakdown)
print(breakdown.pos_score())
print(breakdown.neg_score())
print(breakdown.obj_score())

<work.v.03: PosScore=0.125 NegScore=0.0>
0.125
0.0
0.875
